In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dx, dy = 25, 25
xsize, ysize = 6400, 6400

x = np.arange(dx/2, xsize, dx) - xsize/4
y = np.arange(dy/2, ysize, dy) - ysize/2

In [ ]:
psi = np.zeros((len(y), len(x)))

# R = 200.0
# psi[:, :] = (x[None, :]**2 + y[:, None]**2)**.5 - R # Level set function for circle with radius 200

p = 3.0 # 2.0 is circle, inf is square
a_x, a_y = 200.0, 500.0
psi[:, :] = (np.abs(x[None, :]/a_x)**p + np.abs(y[:, None]/a_y)**p)**(1.0/p) - 1.0
psi_0 = psi.copy()

F = np.ones_like(psi)
dFdt = np.zeros_like(psi)
alpha_F = - 0.003 # 1 / efolding time

H = np.zeros_like(psi)
rf = np.zeros_like(psi)
U = np.zeros_like(psi)

ros = 0.5
alpha_1 = 1.0
u = 3.0 * np.ones_like(psi)
v = 0.0 * np.ones_like(psi)

In [ ]:
time = 0.0
dt = 1.0
total_time = 1800.0

In [ ]:
while time <= total_time:
    grad_psi_x, grad_psi_y = np.gradient(psi, dx, axis=1), np.gradient(psi, dy, axis=0)
    grad_psi = np.hypot(grad_psi_x, grad_psi_y)

    u_norm = grad_psi_x / grad_psi
    v_norm = grad_psi_y / grad_psi

    U = u * u_norm + v * v_norm
    rf = F * ros * np.maximum(0.05, (1.0 + alpha_1 * U)) # The minimum value has a huge impact on result. Do we allow ROS to go below zero-wind?
    
    dpsi_dt = - rf * grad_psi
    psi += dt * dpsi_dt

    dF_dt = np.where(psi < 0.0, alpha_F * F, 0.0)
    F += dt * dF_dt

    H = - dF_dt * 1e7 # Conversion of from fuel fraction to kg / m2 / s to W/m2
    
    time += dt

In [ ]:
plt.figure()
plt.gca().set_aspect("equal")
# plt.pcolormesh(x, y, psi, cmap=plt.cm.Reds)
plt.pcolormesh(x, y, F, cmap=plt.cm.Reds, vmin=0, vmax=1)
plt.colorbar()
#plt.contour(x, y, psi_0, [-1e9, 0, 1e9], colors="k", linestyles=":")
#plt.contour(x, y, psi, [-1e9, 0, 1e9], colors="k")

In [ ]:
plt.figure()
plt.gca().set_aspect("equal")
# plt.pcolormesh(x, y, psi, cmap=plt.cm.Reds)
plt.pcolormesh(x, y, -dF_dt, cmap=plt.cm.turbo)
plt.colorbar()
plt.contour(x, y, psi_0, [-1e9, 0, 1e9], colors="w", linestyles=":")
# plt.contour(x, y, psi, [-1e9, 0, 1e9], colors="k")
plt.xlabel('x (m)')
plt.ylabel('y (m)')

In [ ]:
plt.figure()
plt.gca().set_aspect("equal")
# plt.pcolormesh(x, y, psi, cmap=plt.cm.Reds)
plt.pcolormesh(x, y, rf, cmap=plt.cm.turbo)
plt.colorbar()
plt.contour(x, y, psi_0, [-1e9, 0, 1e9], colors="w", linestyles=":")
# plt.contour(x, y, psi, [-1e9, 0, 1e9], colors="k")
plt.xlabel('x (m)')
plt.ylabel('y (m)')